# Recife — Análise Inicial de Risco de Alagamento e Deslizamento
Este notebook contém um pipeline inicial com dados fictícios para: carregamento, limpeza, análise exploratória (EDA), regressão simples, classificação binária (risco alto/baixo) e um esboço de dashboard interativo com Streamlit.
print(tide_df.head())print('tide_df sample:')---
')

In [ ]:
# Importação das bibliotecas padrão
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve)

# Visual settings
%matplotlib inline
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

## 1) Geração e carregamento de bases fictícias
Geramos `rain_df`, `tide_df` e `vuln_df` com estrutura semelhante ao real.

In [ ]:
neighborhoods = ['Boa Viagem','Espinheiro','Casa Amarela','Ibura','Várzea','Santo Amaro','Centro','Aflitos','Pina','Cordeiro']
date_rng = pd.date_range(start='2024-04-01', end='2024-04-07', freq='6H')

def gen_series(base, scale, size, seed=None):
    rng = np.random.default_rng(seed)
    return np.maximum(0, base + rng.normal(scale=scale, size=size))

rows = []
for nb in neighborhoods:
    for dt in date_rng:
        rows.append({'date': dt, 'neighborhood': nb, 'rain_mm': float(gen_series(base=10 if nb in ['Ibura','Várzea','Pina'] else 3, scale=8, size=1)[0])})
rain_df = pd.DataFrame(rows)

rows = []
for nb in neighborhoods:
    for dt in date_rng:
        base = 120 if nb in ['Boa Viagem','Pina','Aflitos'] else 60
        rows.append({'date': dt, 'neighborhood': nb, 'tide_cm': float(gen_series(base=base, scale=40, size=1)[0])})
tide_df = pd.DataFrame(rows)

vuln_scores = {'Boa Viagem':0.3,'Espinheiro':0.4,'Casa Amarela':0.6,'Ibura':0.8,'Várzea':0.9,'Santo Amaro':0.5,'Centro':0.7,'Aflitos':0.35,'Pina':0.65,'Cordeiro':0.55}
vuln_df = pd.DataFrame([{'neighborhood':k,'vuln_index':v} for k,v in vuln_scores.items()])

print('rain_df sample:')
print(rain_df.head())
print('
---
')
print('tide_df sample:')
print(tide_df.head())
print('
---
')
print('vuln_df:')
print(vuln_df.head())

## 2) Limpeza e preparação básica
Padronizamos datas, juntamos as bases por `date`+`neighborhood`, tratamos ausentes e montamos um `risk_index` heurístico.

In [ ]:
rain_df['date'] = pd.to_datetime(rain_df['date'])
tide_df['date'] = pd.to_datetime(tide_df['date'])
df = pd.merge(rain_df, tide_df, on=['date','neighborhood'], how='outer')
df = pd.merge(df, vuln_df, on='neighborhood', how='left')

df['rain_mm'] = df.groupby('neighborhood')['rain_mm'].apply(lambda x: x.fillna(x.median()))
df['tide_cm'] = df.groupby('neighborhood')['tide_cm'].apply(lambda x: x.fillna(x.median()))
df['vuln_index'] = df['vuln_index'].fillna(df['vuln_index'].median())

df['rain_norm'] = df['rain_mm'] / df['rain_mm'].max()
df['tide_norm'] = df['tide_cm'] / df['tide_cm'].max()
df['vuln_norm'] = df['vuln_index']
df['risk_index'] = 0.5*df['rain_norm'] + 0.3*df['tide_norm'] + 0.2*df['vuln_norm']
df['risk_label'] = (df['risk_index'] > 0.5).astype(int)

df.head()

## 3) Análise Exploratória (EDA)
Histograma da chuva, boxplot de maré por bairro e mapa de calor de correlações.

In [ ]:
plt.figure(figsize=(10,5))
sns.histplot(df['rain_mm'], bins=30, kde=True, color='blue')
plt.title('Distribuição da chuva (mm)')
plt.xlabel('mm de chuva')
plt.ylabel('contagem')
plt.show()

plt.figure(figsize=(12,6))
order = df.groupby('neighborhood')['tide_cm'].median().sort_values().index
sns.boxplot(data=df, x='neighborhood', y='tide_cm', order=order)
plt.xticks(rotation=45)
plt.title('Boxplot: maré (cm) por bairro')
plt.show()

corr_df = df[['rain_mm','tide_cm','vuln_index','risk_index']].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr_df, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Mapa de calor: correlação entre variáveis')
plt.show()

## 4) Regressão linear simples: chuva -> risk_index
Regressão linear, scatter com linha de tendência e análise de resíduos.

In [ ]:
X = df[['rain_mm']].values
y = df['risk_index'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

plt.figure(figsize=(8,6))
plt.scatter(X_test, y_test, label='observado', alpha=0.6)
xs = np.linspace(X.min(), X.max(), 100).reshape(-1,1)
plt.plot(xs, lr.predict(xs), color='red', label='linha tendência')
plt.xlabel('rain_mm')
plt.ylabel('risk_index')
plt.legend()
plt.title('Regressão linear: chuva -> risk_index')
plt.show()

residuals = y_test - y_pred
plt.figure(figsize=(8,4))
sns.histplot(residuals, kde=True)
plt.title('Distribuição dos resíduos')
plt.show()

print('Coeficiente (slope):', lr.coef_[0])
print('Intercept:', lr.intercept_)
print('R^2 (treino):', lr.score(X_train, y_train))
print('R^2 (teste):', lr.score(X_test, y_test))

## 5) Classificação simples
LogisticRegression para classificar risco alto vs baixo; mostramos matriz de confusão, métricas e curvas ROC/PR.

In [ ]:
features = ['rain_mm','tide_cm','vuln_index']
X = df[features].values
y = df['risk_label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:,1]

cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)

print('Confusion matrix:
', cm)
print(f'Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1: {f1:.3f}')

fpr, tpr, _ = roc_curve(y_test, y_proba)
auc = roc_auc_score(y_test, y_proba)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'ROC (AUC = {auc:.2f})')
plt.plot([0,1],[0,1],'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

prec_vals, rec_vals, _ = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(6,5))
plt.plot(rec_vals, prec_vals)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.show()

## 6) Esboço de dashboard Streamlit
Exemplo de script (salvar como `app.py`). Instruções e próximos passos estão no final do notebook.

In [ ]:
streamlit_example = '''
import streamlit as st
import pandas as pd
import altair as alt

st.title('Recife — Painel de Risco: Alagamento e Deslizamento')
st.sidebar.header('Filtros')
neighborhood = st.sidebar.selectbox('Selecione o bairro', options=df['neighborhood'].unique())
var = st.sidebar.selectbox('Escolha a variável', options=['rain_mm','tide_cm','vuln_index','risk_index'])
df_nb = df[df['neighborhood'] == neighborhood].sort_values('date')
chart = alt.Chart(df_nb).mark_line().encode(x='date:T', y=alt.Y(var+':Q'), tooltip=['date','rain_mm','tide_cm','vuln_index','risk_index']).properties(width=700, height=350)
st.altair_chart(chart)
'''

## Comentários finais e próximos passos
- Melhorias: janelas temporais para chuva, normalização por bairro, features adicionais (uso do solo, drenagem), validação com eventos reais e deploy no Streamlit Cloud.
- Entregáveis: notebook, `app.py` streamlit, README com plano do dashboard.